In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets 
# preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.metrics import mean_squared_error, roc_auc_score
import time

%matplotlib inline
import seaborn as sns
sns.set_theme()

In [ ]:
train_dir = "/kaggle/input/commonlitreadabilityprize"
train_path = os.path.join(train_dir, "train.csv")
test_path = os.path.join(train_dir, "test.csv")

sample = os.path.join(train_dir, "sample_submission.csv")
df = pd.read_csv(train_path)
df.head()

df_test = pd.read_csv(test_path)

df_sample = pd.read_csv(sample)
df_test

In [ ]:
def plot_graph(data):
    tar_s = np.unique(data['target']).shape

    fig,ax = plt.subplots(1,2,figsize=(15,8))
    sns.scatterplot(x = np.arange(tar_s[0]), y =data['target'],ax=ax[0] )
    sns.scatterplot(x = np.arange(tar_s[0]), y =data['standard_error'],ax=ax[1] )
    return fig, ax
plot_graph(df)
    # A outlier could easily be drop with IQR and pandas dataframe filter method
    # There is one outlier in standard error at index = 0

In [ ]:
def get_validate_data(X, y):
    X_train, X_test, y_train,y_test = train_test_split(X, y, test_size= 0.2)
    print(X_train.shape, X_test.shape, y_train.shape,y_test.shape)
    return X_train, X_test, y_train,y_test


In [ ]:
Q3 = df_describe.loc['75%']; Q1 = df_describe.loc['25%']
IQR = Q3 - Q1

upper_iqr = Q3 +1.5*IQR
lower_iqr  = Q1 - 1.5*IQR

# print(f"Condition for outliers is {df['target']<=  upper_iqr and df['target']<= lower_iqr}")
df['target']
m = map(lambda x: x>= lower_iqr, df['target'])


df_upper = df.groupby("target").filter(lambda x: (x['standard_error']> upper_iqr['standard_error'] ))
df_lower = df.groupby("target").filter(lambda x: (x['standard_error']< lower_iqr['standard_error'] ))

df_upper2 = df.groupby("standard_error").filter(lambda x: (x['target']> upper_iqr['target'] ))
df_lower2 = df.groupby("standard_error").filter(lambda x: (x['target']< lower_iqr['target'] ))

df.drop(index= df_upper.index, axis=0, inplace=True)
df.drop(index = df_lower.index, axis=0, inplace=True)
df.drop(index = df_upper2.index, axis=0, inplace=True)
df.drop(index = df_lower2.index, axis=0, inplace=True)


In [ ]:
plot_graph(df)

In [ ]:
def rmse_score(model, X, y):
    y_pred = model.predict(X)
    score = cross_val_score(model, X, y, n_jobs= -1, cv= 5); m = mean_squared_error(y, y_pred)
    print(f"Cross_val_Score for X_train prediction:\t{score}\nmean squared error:\t{m}")
    return score, m 

In [ ]:
import re

pat = re.compile("!@#$%^&*()_+<>[?]")
def pre(data):
    data =data.replace("!@#$%^&*()_+<>[?]","")
    return data
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
d = vectorizer.fit_transform(df['excerpt'])


In [ ]:
y_train = df['target']
X_train = d.toarray()
X_train.shape, y_train.shape

X_train, X_test, y_train,y_test = get_validate_data(X_train, y_train)


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor

model = GaussianProcessRegressor(n_restarts_optimizer= 10 )

model.fit(X_train, y_train)

In [ ]:
test_score, test_mean = rmse_score(model, X_test,y_test)
train_score, train_mean = rmse_score(model, X_train,y_train)


In [ ]:
fig, ax = plt.subplots(1,2, figsize= (15, 8))
ax[0].plot(test_score, 'y')
ax[0].plot(train_score, 'r')
ax[0].set_title("cross_val_score")

ax[0].annotate("Validation data",xy= (2.0,0.410), arrowprops=dict(facecolor='red', shrink=5),fontsize=16,horizontalalignment='left')
ax[0].annotate("training data",xy= (2.0,0.30), arrowprops=dict(facecolor='red', shrink=5),fontsize=16,horizontalalignment='left')

# from graph, it is clear, our model is overfitting data very high, could be fix with l1 or l2 

In [ ]:
# gradient boosting regression 
x = time.time()
params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'
         }

from sklearn.ensemble import GradientBoostingRegressor

grad_estimator = GradientBoostingRegressor(**params)

grad_estimator.fit(X_train, y_train)

g_score, g_mean = rmse_score(grad_estimator, X_test, y_test)

print("completion time")
print(time.time()-x)

In [ ]:
g_score_train, g_mean_train = rmse_score(grad_estimator, X_train, y_train)
g_score,g_mean

In [ ]:
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(grad_estimator.staged_predict(X_test)):
    test_score[i] = grad_estimator.loss_(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1,1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, grad_estimator.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')
fig.tight_layout()
fig

In [ ]:
def predict_from_gauss(data, num):
    p = data.iloc[num]['excerpt']

    p = vectorizer.transform([p])
    pred = model.predict(p.toarray())
    print(pred)
    return pred[0]


In [ ]:
g_score, g_mean

In [ ]:
df_test['target']=0

for i in range(df_test.shape[0]):
    df_test.iloc[i, -1] = predict_from_gauss(df_test, i)


In [ ]:
for i in ['url_legal','license','excerpt']:
    try:
        df_test.drop(i, inplace=True, axis=1)
    except KeyError:
        pass

In [ ]:
df_test.to_csv("submission.csv", index=False)
